Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 17930 images belonging to 12 classes.
Found 4477 images belonging to 12 classes.
Số class: 12
CLASS_NAMES = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy', 'Blueberry___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_', 'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot', 'Grape___Esca_(Black_Measles)', 'Grape___healthy']
✅ Đang load model cũ để fine-tune...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,589,004 (9.88 MB)

 Trainable params: 331,020 (1.26 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/5
561/561 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.9610 - loss: 0.1133

561/561 ━━━━━━━━━━━━━━━━━━━━ 4294s 8s/step - accuracy: 0.9610 - loss: 0.1133 - val_accuracy: 0.9741 - val_loss: 0.0744
Epoch 2/5
561/561 ━━━━━━━━━━━━━━━━━━━━ 1319s 2s/step - accuracy: 0.9642 - loss: 0.1057 - val_accuracy: 0.9685 - val_loss: 0.0868
Epoch 3/5
561/561 ━━━━━━━━━━━━━━━━━━━━ 1338s 2s/step - accuracy: 0.9695 - loss: 0.0876 - val_accuracy: 0.9727 - val_loss: 0.0698
Epoch 4/5
561/561 ━━━━━━━━━━━━━━━━━━━━ 1333s 2s/step - accuracy: 0.9663 - loss: 0.0968 - val_accuracy: 0.9732 - val_loss: 0.0748
Epoch 5/5
561/561 ━━━━━━━━━━━━━━━━━━━━ 1330s 2s/step - accuracy: 0.9705 - loss: 0.0836 - val_accuracy: 0.9701 - val_loss: 0.0693


2025-09-28 14:32:31.279049: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759069951.318765   56101 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759069951.331419   56101 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759069951.379016   56101 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759069951.379075   56101 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759069951.379083   56101 computation_placer.cc:177] computation placer alr

In [ ]:
# 1️⃣ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2️⃣ Cài thư viện
!pip install tensorflow matplotlib tensorflowjs --quiet

# 3️⃣ Import
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# 4️⃣ Đường dẫn dataset & model
DATASET_PATH = '/content/drive/MyDrive/dataset'         # dataset đầy đủ
MODEL_PATH = '/content/drive/MyDrive/plant_model.keras' # model chuẩn
TFJS_PATH   = '/content/drive/MyDrive/plant_model_js'   # thư mục xuất cho web

# 5️⃣ Tham số
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS_NEW = 10   # train mới
EPOCHS_FINE = 5   # fine-tune nhanh

# 6️⃣ Data generator
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.2
)

train_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

num_classes = len(train_gen.class_indices)
print("📌 Số lớp:", num_classes)
print("📌 CLASS_NAMES =", list(train_gen.class_indices.keys()))

# 7️⃣ Nếu có model cũ thì fine-tune nhanh
if os.path.exists(MODEL_PATH):
    print("✅ Đang load model cũ để fine-tune...")
    model = load_model(MODEL_PATH)

    # cho phép train lại block cuối cùng để học thêm ảnh mới
    for layer in model.layers[-20:]:
        if not isinstance(layer, Dropout):
            layer.trainable = True

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    checkpoint = ModelCheckpoint(MODEL_PATH, save_best_only=True, monitor='val_accuracy', mode='max')
    history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS_FINE, callbacks=[checkpoint])

else:
    print("🚀 Không tìm thấy model cũ. Train model mới từ đầu...")
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    checkpoint = ModelCheckpoint(MODEL_PATH, save_best_only=True, monitor='val_accuracy', mode='max')
    history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS_NEW, callbacks=[checkpoint])

# 8️⃣ Vẽ đồ thị
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')
plt.show()

# 9️⃣ Convert sang TF.js
!rm -rf {TFJS_PATH}   # xoá thư mục cũ nếu có
!tensorflowjs_converter --input_format keras {MODEL_PATH} {TFJS_PATH}
print("✅ TensorFlow.js model saved to:", TFJS_PATH)

# 🔟 Xuất class names để copy sang web
print("📌 CLASS_NAMES =", list(train_gen.class_indices.keys()))


In [8]:
from tensorflow.keras.models import load_model

# Load model .h5
model = load_model("/content/drive/MyDrive/plant_model.h5")

# Save lại sang định dạng .keras
model.save("/content/drive/MyDrive/plant_model.keras")
